In [0]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import os

In [2]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = keras.utils.get_file(name, origin=DIRECTORY_URL+name)

parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - 0s 0us/step


'/root/.keras/datasets'

In [0]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [0]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [0]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [7]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'pursued the Trojans and Lycians in the pride and foolishness of his'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"a council of war, and determine to send scouts into the enemy's camp,">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Approach--thou shalt be taught with good effect'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Yet never heard I once hard speech from thee,'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"And check'd his forward course; then rushing on">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


In [8]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [10]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'pursued the Trojans and Lycians in the pride and foolishness of his'


In [11]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[6994, 664, 12379, 15237, 8614, 930, 664, 5018, 15237, 8163, 15383, 944]


In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [0]:
def encode_map_fn(text, label):
  encoded_text, label = tf.py_function(encode,
                                       inp=[text, label],
                                       Tout=(tf.int64, tf.int64))
  
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label

all_encoded_data = all_labeled_data.map(encode_map_fn)

In [0]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [15]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(16,), dtype=int64, numpy=
 array([ 6994,   664, 12379, 15237,  8614,   930,   664,  5018, 15237,
         8163, 15383,   944,     0,     0,     0,     0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>)

In [0]:
vocab_size += 1

In [0]:
model = keras.Sequential()

In [0]:
model.add(keras.layers.Embedding(vocab_size, 64))

In [0]:
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64)))

In [0]:
for units in [64, 64]:
  model.add(keras.layers.Dense(units, activation='relu'))

model.add(keras.layers.Dense(3))

In [0]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [22]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================] - 19s 27ms/step - loss: 0.5182 - accuracy: 0.7516 - val_loss: 0.3841 - val_accuracy: 0.8254
Epoch 2/3
697/697 [==============================] - 18s 26ms/step - loss: 0.3016 - accuracy: 0.8665 - val_loss: 0.3466 - val_accuracy: 0.8380
Epoch 3/3
697/697 [==============================] - 18s 26ms/step - loss: 0.2246 - accuracy: 0.9029 - val_loss: 0.3813 - val_accuracy: 0.8262


In [23]:
eval_loss, eval_acc = model.evaluate(test_data)

print(f"Eval loss: {eval_loss}, Eval accuracy: {eval_acc}")

79/79 [==============================] - 2s 28ms/step - loss: 0.3813 - accuracy: 0.8262
Eval loss: 0.38125818967819214, Eval accuracy: 0.826200008392334
